In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
test=pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
train.isnull().sum().sum()

In [ ]:
test.isnull().sum().sum()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.Policy_Sales_Channel.value_counts()

In [ ]:
train.Vintage.value_counts()

In [ ]:
train['Gender'] = train['Gender'].map({"Male":1,"Female":0}).astype(int)

In [ ]:
train = pd.get_dummies(train, drop_first=True)

In [ ]:
train.head()

In [ ]:
train=train.rename(columns={"Vehicle_Age_< 1 Year": "Vehicle_Age_lt_1_Year", "Vehicle_Age_> 2 Years": "Vehicle_Age_gt_2_Years"})
train['Vehicle_Age_lt_1_Year']=train['Vehicle_Age_lt_1_Year'].astype('int')
train['Vehicle_Age_gt_2_Years']=train['Vehicle_Age_gt_2_Years'].astype('int')
train['Vehicle_Damage_Yes']=train['Vehicle_Damage_Yes'].astype('int')

In [ ]:
num_feat = ['Age','Vintage']

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, RobustScaler

In [ ]:
ss = StandardScaler()

In [ ]:
train[num_feat] = ss.fit_transform(train[num_feat])

In [ ]:
train.head()

In [ ]:
mm = MinMaxScaler()
train[['Annual_Premium']] = mm.fit_transform(train[['Annual_Premium']])

In [ ]:
train.describe()

In [ ]:
train.head()

In [ ]:
cat_feat = ['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age_lt_1_Year','Vehicle_Age_gt_2_Years','Vehicle_Damage_Yes','Region_Code','Policy_Sales_Channel']

In [ ]:
for column in cat_feat:
    train[column] = train[column].astype('str')

In [ ]:
test['Gender'] = test['Gender'].map( {'Female': 0, 'Male': 1} ).astype(int)
test=pd.get_dummies(test,drop_first=True)
test=test.rename(columns={"Vehicle_Age_< 1 Year": "Vehicle_Age_lt_1_Year", "Vehicle_Age_> 2 Years": "Vehicle_Age_gt_2_Years"})
test['Vehicle_Age_lt_1_Year']=test['Vehicle_Age_lt_1_Year'].astype('int')
test['Vehicle_Age_gt_2_Years']=test['Vehicle_Age_gt_2_Years'].astype('int')
test['Vehicle_Damage_Yes']=test['Vehicle_Damage_Yes'].astype('int')

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
ss = StandardScaler()
test[num_feat] = ss.fit_transform(test[num_feat])


mm = MinMaxScaler()
test[['Annual_Premium']] = mm.fit_transform(test[['Annual_Premium']])

In [ ]:
for column in cat_feat:
    test[column] = test[column].astype('str')

In [ ]:
train=train.drop('id',axis=1)

In [ ]:
train.columns

In [ ]:
from sklearn.model_selection import train_test_split

train_target = train['Response']
train = train.drop(['Response'], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(train,train_target,random_state=0)

In [ ]:
id=test.id
test=test.drop('id',axis=1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, GridSearchCV, KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_search = {'criterion':['entropy','gini'],
                'max_depth':[2,3,4,5,6,7,10],
                'min_samples_leaf':[4,6,8],
                'min_samples_split':[5,7,10],
                'n_estimators':[300]}

clf = RandomForestClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10, 
                               cv = 4, verbose= 1, random_state= 101, n_jobs = -1)
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report 

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
y_test.shape

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score,roc_curve
y_score = model.predict_proba(x_test)[:,1]
fpr,tpr,_ = roc_curve(y_test,y_score)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import auc

plt.title('Random Forest ROC curve: CC Fraud')
plt.xlabel('FPR (Precision)')
plt.ylabel('TPR (Recall)')

plt.plot(fpr,tpr)
plt.plot((0,1), ls='dashed', color='green')
plt.show()
print ('Area under curve (AUC): ', auc(fpr,tpr))

In [ ]:
roc_auc_score(y_test, y_score)

In [ ]:
id

In [ ]:
test.head()

In [ ]:
Preds = [pred[1] for pred in model.predict_proba(test)]

In [ ]:
submission = pd.DataFrame(data = {'id': id, 'Response': Preds})
submission.head()

In [ ]:
submission.describe()

In [ ]:
submission.to_csv('vehicle_insurance.csv', index = False)